In [1]:
import boto3
import json
import numpy as np
import pandas as pd
import time

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [2]:
iam = boto3.client("iam")
s3 = boto3.client("s3")
bucket = "recommend-bucket-epicmobile"
filename = "testdata.csv"

In [3]:
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data = data[data['RATING'] > 3]                # Keep only movies rated higher than 3 out of 5.
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

In [5]:
#데이터 셋 만들기
create_dataset_group_response = personalize.create_dataset_group(
    name = "recommend-dataset-epicmobile"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#데이터 셋 활성상태 기다리기
max_time = time.time() + 3*60*60 #3시간
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:570872761770:dataset-group/recommend-dataset-epicmobile",
  "ResponseMetadata": {
    "RequestId": "f1746e83-a03c-4822-9dff-1effcdebae57",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 Jan 2021 04:18:06 GMT",
      "x-amzn-requestid": "f1746e83-a03c-4822-9dff-1effcdebae57",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}
DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


In [8]:
#상호작용 데이터 셋 만들기
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "recommend-interdataset-epicmobile",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:570872761770:dataset/recommend-dataset-epicmobile/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "e8ea21b1-28e7-41d9-9d4c-e9302956fbff",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 Jan 2021 04:44:45 GMT",
      "x-amzn-requestid": "e8ea21b1-28e7-41d9-9d4c-e9302956fbff",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [12]:
#s3버킷 정책 연결
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

#personalize 정책
iam = boto3.client("iam")

role_name = "Personalize-role-epicmobile"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60)

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::570872761770:role/Personalize-role-epicmobile


In [13]:
#s3데이터 personalize로 가져오기
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-import-epicmobile",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#데이터 셋 활성상태 기다림
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:570872761770:dataset-import-job/personalize-import-epicmobile",
  "ResponseMetadata": {
    "RequestId": "098a90e1-87b1-49ea-b24e-45d7f3216ffe",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 Jan 2021 04:49:45 GMT",
      "x-amzn-requestid": "098a90e1-87b1-49ea-b24e-45d7f3216ffe",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}
DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
Dataset

In [14]:
#솔루션 만들기
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" #user-personaliztion 사용해서 사용자에 맞는 추천리스트 출력

create_solution_response = personalize.create_solution(
    name = "personalize-solution-epicmobile",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#솔루션 활성상태 기다림(학습중)
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#솔루션 버전의 지표가져오기
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:570872761770:solution/personalize-solution-epicmobile",
  "ResponseMetadata": {
    "RequestId": "b193c7ac-9ce2-4f27-8f7f-b3e1c0b2c1d6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 Jan 2021 05:04:59 GMT",
      "x-amzn-requestid": "b193c7ac-9ce2-4f27-8f7f-b3e1c0b2c1d6",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}
{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:570872761770:solution/personalize-solution-epicmobile/9ae63f1f",
  "ResponseMetadata": {
    "RequestId": "35035f71-03a6-44db-911b-6f0ad55b2fea",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 Jan 2021 05:04:59 GMT",
      "x-amzn-requestid": "35035f71-03a6-44db-911b-6f0ad55b2fea",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAtt

In [18]:
#켐페인 만들기
create_campaign_response = personalize.create_campaign(
    name = "personalize-campaign-epicmobile",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#켐페인 활성상태 기다림
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateCampaign operation: Another resource with Arn arn:aws:personalize:us-east-1:570872761770:campaign/personalize-campaign-epicmobile already exists.